In [1]:
import os
%pwd

'c:\\Trenches\\Text-Summarizer\\research'

In [2]:
os.chdir('../')
%pwd

'c:\\Trenches\\Text-Summarizer'

In [3]:
#Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int


In [4]:
#Configuration manager
from txtSummarizer.constants import *
from txtSummarizer.utils.common import read_yaml, create_directory

In [5]:
#Configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])


    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        params = self.params.TrainingArguments

        create_directory([config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir = config.root_dir,
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            num_train_epochs= params.num_train_epochs,
            warmup_steps= params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            weight_decay= params.weight_decay,
            logging_steps= params.logging_steps,
            evaluation_strategy= params.evaluation_strategy,
            eval_steps= params.eval_steps,
            save_steps= params.save_steps,
            gradient_accumulation_steps= params.gradient_accumulation_steps,
        )

        return model_training_config

In [6]:
#Component
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\user\miniconda3\envs\txt_sum\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def train(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        pegasus_model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=pegasus_model)


        #collating the data
        train_dataset = load_from_disk(self.config.data_path)

        training_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model=pegasus_model, 
            args=training_args, 
            tokenizer=tokenizer, 
            data_collator=seq2seq_data_collator,
            train_dataset=train_dataset['train'], 
            eval_dataset=train_dataset['validation']
        )

        trainer.train()

        #save the model
        pegasus_model.save_pretrained(os.path.join(self.config.root_dir, "samsum-pegasus-model"))
        #save the tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))     


In [8]:
from txtSummarizer.logging import logger

In [14]:
#pipeline
try:
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_training_pipeline = ModelTraining(config=model_training_config)
    model_training_pipeline.train()
except Exception as e:
    logger.error(f"model training pipeline failed with error: {e}")
    raise e

[2023-08-17 23:01:27,926: INFO: common: Config: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/ladiyusuph/Text-Summarizer/raw/main/samsum_data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'ALL_REQUIRED_FILES': ['train', 'test', 'validation']}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/samsum_dataset', 'tokenizer_name': 'google/pegasus-cnn_dailymail'}, 'model_training': {'root_dir': 'artifacts/model_training', 'data_path': 'artifacts/data_transformation/samsum_dataset', 'model_ckpt': 'google/pegasus-cnn_dailymail'}}]
[2023-08-17 23:01:28,100: INFO: common: Config: {'TrainingArguments': {'num_train_epochs': 1, 'warmup_steps': 500, 'per_device_train_batch_size': 1, 'weight_de

[2023-08-17 23:02:17,600: ERROR: 2051896027: model training pipeline failed with error: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /google/pegasus-cnn_dailymail/resolve/main/tokenizer_config.json (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1131)')))"), '(Request ID: 0aac82f7-c9a0-4e24-a87c-7d7ab6f78850)')]


SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /google/pegasus-cnn_dailymail/resolve/main/tokenizer_config.json (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1131)')))"), '(Request ID: 0aac82f7-c9a0-4e24-a87c-7d7ab6f78850)')